In [1]:
# Week 2: Model Training and Validation

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import sqlite3
import pandas as pd



In [2]:
# Paths to the dataset directories
train_dir = os.path.join('student_images', 'train')
val_dir = os.path.join('student_images', 'validation')
test_dir = os.path.join('student_images', 'test')

In [3]:
# Image size
image_size = (224, 224)

In [4]:
# Function to load images and labels from a directory
def load_images(data_dir, image_size=(224, 224)):
    images = []
    labels = []
    for label in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, label)
        if os.path.isdir(class_dir):
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)
                images.append(image)
                labels.append(int(label))
    return np.array(images), np.array(labels)

In [5]:
# Load datasets
X_train, y_train = load_images(train_dir)
X_val, y_val = load_images(val_dir)

In [6]:
# Reduce image size
resize_shape = (112, 112)  # Adjust as needed

X_train_resized = np.array([cv2.resize(img, resize_shape) for img in X_train])
X_val_resized = np.array([cv2.resize(img, resize_shape) for img in X_val])

# Normalize images
X_train_resized = X_train_resized / 255.0
X_val_resized = X_val_resized / 255.0

In [7]:
import gc

# Clear cache and force garbage collection
gc.collect()


0

In [8]:
# Define the ImageDataGenerator for training with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Define the ImageDataGenerator for validation without augmentation
val_datagen = ImageDataGenerator(rescale=1./255)

# Define directories (adjust paths as needed)
current_dir = os.getcwd()
train_dir = os.path.join(current_dir, 'student_images', 'train')
val_dir = os.path.join(current_dir, 'student_images', 'validation')

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 10512 images belonging to 18 classes.
Found 4303 images belonging to 18 classes.


In [9]:
# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(18, activation='softmax'),
    Dense(18, activation='softmax')
])

C:\Users\DELL\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_val, y_val))

In [ ]:
# Model training with the data generators
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Adjust number of epochs as needed
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

Epoch 1/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 756s 2s/step - accuracy: 0.2262 - loss: 2.6805 - val_accuracy: 0.2750 - val_loss: 2.4490
Epoch 2/10
  1/328 ━━━━━━━━━━━━━━━━━━━━ 10:24 2s/step - accuracy: 0.3438 - loss: 2.4763

C:\Users\DELL\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 993us/step - accuracy: 0.3438 - loss: 1.2419 - val_accuracy: 0.2667 - val_loss: 1.2330
Epoch 3/10
 76/328 ━━━━━━━━━━━━━━━━━━━━ 9:03 2s/step - accuracy: 0.2614 - loss: 2.5078

In [ ]:
# Save the model
model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'facial_recognition_model.keras')
model.save(model_path)

In [ ]:
# Week 3: Real-time Recognition and Attendance Marking

# Load the trained model
model = load_model(model_path)

In [ ]:
# Load student records from SQLite database
def load_student_records(db_path):
    conn = sqlite3.connect(db_path)
    students_db = pd.read_sql_query('SELECT * FROM students', conn)
    conn.close()
    return students_db

In [ ]:
# Load the student records
students_db = load_student_records('student_attendance.db')

In [ ]:
# Print student records to verify
print(students_db)

In [ ]:
# Real-time recognition function
def recognize_and_display():
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        
        # Check if the image path exists
        if not os.path.isfile(img_path):
            print(f"Image file '{img_path}' does not exist.")
            continue
        
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # Check if the image was successfully read
        if img is None:
            print(f"Failed to read image '{img_path}'.")
            continue
        
        faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        for (x, y, w, h) in faces:
            face = img[y:y+h, x:x+w]
            face = cv2.resize(face, (224, 224))
            face = face / 255.0
            face = np.expand_dims(face, axis=0)
            
            predictions = model.predict(face)
            predicted_label = np.argmax(predictions[0])
            
            student_name = students_db.loc[students_db['student_id'] == predicted_label, 'student_name'].values[0]
            
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(img, student_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        
        plt.imshow(img, cmap='gray')
        plt.title(f"Image: {img_name}")
        plt.axis('off')
        plt.show()

# Call the function
recognize_and_display()